## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [7]:
import os
import keras

# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

In [10]:
from keras.datasets import cifar10
from resnet_builder import resnet
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense, Dropout

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical (y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [12]:
from sklearn.utils import shuffle
def my_generator(x, y, batch_size):
    while True:
        for idx in range(0, len(x), batch_size): # 讓 idx 從 0 開始，一次增加 batch size。假設 batch_size=32, idx = 0, 32, 64, 96, ....
            batch_x, batch_y = x[idx:idx+batch_size], y[idx:idx+batch_size]
            yield batch_x, batch_y
        x, y = shuffle(x, y) # loop 結束後，將資料順序打亂再重新循環

In [23]:
batch_size = 64
epochs = 10
train_generator = my_generator(x_train, y_train, batch_size) # 建立好我們寫好的 generator

In [20]:
model = resnet(input_shape=(32,32,3))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_63 (Conv2D)              (None, 32, 32, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 32, 32, 16)   64          conv2d_63[0][0]                  
__________________________________________________________________________________________________
activation_57 (Activation)      (None, 32, 32, 16)   0           batch_normalization_57[0][0]     
____________________________________________________________________________________________

In [19]:
"""
FREEZE_LAYERS = 2

model = ResNet50(include_top=False, weights='imagenet', input_tensor=None,input_shape=(32,32,3))
x = model.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(10, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
model_final = Model(inputs=model.input, outputs=output_layer)
for layer in model_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
model_final.compile(optimizer=Adam(lr=1e-5),
                    loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(model_final.summary())
"""

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [24]:
# 訓練模型
model.fit_generator(train_generator,
                    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
781/781 [==============================] - 1913s 2s/step - loss: 1.7031 - accuracy: 0.5544 - val_loss: 1.6457 - val_accuracy: 0.5667
Epoch 2/10
781/781 [==============================] - 1878s 2s/step - loss: 1.2952 - accuracy: 0.6737 - val_loss: 1.3590 - val_accuracy: 0.6451
Epoch 3/10
781/781 [==============================] - 1701s 2s/step - loss: 1.0987 - accuracy: 0.7282 - val_loss: 1.3479 - val_accuracy: 0.6430
Epoch 4/10
781/781 [==============================] - 1739s 2s/step - loss: 0.9695 - accuracy: 0.7731 - val_loss: 1.1594 - val_accuracy: 0.7132
Epoch 5/10
781/781 [==============================] - 1743s 2s/step - loss: 0.8732 - accuracy: 0.8037 - val_loss: 1.1454 - val_accuracy: 0.7040
Epoch 6/10
781/781 [==============================] - 1727s 2s/step - loss: 0.8060 - accuracy: 0.8261 - val_loss: 1.0415 - val_accuracy: 0.7558
Epoch 7/10
781/781 [==============================] - 1701s 2s/step - loss: 0.7530 - accuracy: 0.8437 - val_loss: 1.0917 - val_accuracy: